# Content-based Filtering Model Inference

In [1]:
import csv
import pandas as pd
import tensorflow as tf

In [2]:
brand_choice_list = ['Oppo', 'Asus', 'Infinix', 'Samsung', 'Vivo', 'Huawei', 'Apple', 'Realme', 'Xiaomi', 'Poco', 'lainnya/tidak ada']

In [3]:
user_model = tf.keras.models.load_model('user_model.keras')

Preprocess data into input compatible with the model.

In [4]:
def preprocess_user(x):
  one_hot = x[-1]
  one_hot = tf.one_hot(brand_choice_list.index(one_hot), len(brand_choice_list))
  x = tf.cast(tf.convert_to_tensor(x[:-1]), tf.float32)
  return tf.expand_dims(tf.concat([x, one_hot], 0), 0)

In [5]:
a = [5,5,5,5,5,5,'Apple'] # performance, camera, battery, software, RAM, storage, favorite brand
preprocessed_output = preprocess_user(a)

In [6]:
u_vector = user_model.predict(preprocessed_output)

1/1 [==============================] - 0s 143ms/step


In [7]:
p_vector = pd.read_csv('phone_vector.csv', header=None)

In [8]:
p_vector = tf.convert_to_tensor(p_vector)
p_vector = tf.cast(p_vector, tf.float32)

Predict phone ratings for a user.

In [9]:
tf.tensordot(u_vector, p_vector, axes=(1,1)) * 5

<tf.Tensor: shape=(1, 96), dtype=float32, numpy=
array([[4.1724195, 4.173358 , 4.067517 , 4.1683087, 4.053091 , 4.2440376,
        4.2785506, 4.3623433, 4.071537 , 3.9885628, 4.3750887, 4.0470743,
        3.9292228, 3.9391234, 3.852385 , 4.3369503, 4.0721397, 4.340616 ,
        4.3350215, 3.9016051, 4.031767 , 4.237111 , 4.130212 , 4.0591235,
        4.0229473, 4.072178 , 4.171513 , 4.1101274, 4.1462283, 4.1123266,
        4.409349 , 4.389525 , 4.416095 , 4.4271855, 4.3020277, 4.1115675,
        4.060865 , 4.101839 , 4.1523294, 4.175838 , 3.9868023, 4.0394287,
        4.340296 , 3.842646 , 3.9561892, 3.9123821, 3.751477 , 4.0252056,
        3.786425 , 3.930696 , 4.3380375, 4.265899 , 4.304504 , 3.904066 ,
        3.9667754, 4.2945714, 4.2981772, 4.041826 , 3.9962647, 3.9836729,
        3.7505097, 4.1413608, 3.6951737, 3.9945638, 4.140338 , 4.428541 ,
        4.428343 , 4.111965 , 4.1107254, 4.428527 , 4.428324 , 3.9347343,
        3.931728 , 3.9260297, 3.9245448, 3.8975825, 3.9268193, 

In [10]:
rating_pred_list = []
for p in p_vector:
  rating_pred_list.append(tf.tensordot(u_vector, tf.expand_dims(p, 0), axes=(1,1)) * 5)

Get phone name.

In [11]:
phone_pred_list = pd.read_csv('phone_dataset.csv')

In [12]:
phone_pred_list = phone_pred_list['name'].tolist()

Sort ratings to get top 10 recommended phones for a user.

In [13]:
top_10_phone = list(zip(rating_pred_list, phone_pred_list))
top_10_phone.sort(reverse=True)
rating_pred_list, phone_pred_list = zip(*top_10_phone)
rating_pred_list, phone_pred_list = list(rating_pred_list), list(phone_pred_list)

In [14]:
top_10_list = []
for i in range(10):
  top_10_list.append(phone_pred_list[i])
  print(i+1, phone_pred_list[i], rating_pred_list[i].numpy())

1 iPhone 14 Pro [[4.428541]]
2 iPhone 15 Pro [[4.428527]]
3 iPhone 14 Pro Max [[4.428343]]
4 iPhone 15 Pro Max [[4.428324]]
5 Samsung Galaxy Z Fold5 [[4.4271855]]
6 Samsung Galaxy S24 [[4.416095]]
7 Samsung Galaxy S24 Ultra [[4.409349]]
8 Samsung Galaxy S24+ [[4.389525]]
9 Oppo Find N3 [[4.3750887]]
10 Oppo Find X7 [[4.3623433]]


In [15]:
top_10_list

['iPhone 14 Pro',
 'iPhone 15 Pro',
 'iPhone 14 Pro Max',
 'iPhone 15 Pro Max',
 'Samsung Galaxy Z Fold5',
 'Samsung Galaxy S24',
 'Samsung Galaxy S24 Ultra',
 'Samsung Galaxy S24+',
 'Oppo Find N3',
 'Oppo Find X7']